In [1]:
import matplotlib 
%matplotlib auto
import matplotlib.pyplot as plt
from astropy.io.votable import parse_single_table
import astropy.units as u
from astropy.coordinates import SkyCoord
from astropy import units as u
from astropy.io import fits
import numpy as np
import pandas as pd
import glob
from sklearn.cluster import KMeans
from sklearn.cluster import DBSCAN
from sklearn import metrics
import scipy
from scipy.stats import zscore
from scipy.spatial import distance
from __future__ import division
from sklearn.preprocessing import StandardScaler
import hdbscan
# from astroquery.gaia import Gaia
import seaborn as sns
from mpl_toolkits.axes_grid1.anchored_artists import AnchoredSizeBar
from matplotlib.patches import Rectangle
import scipy.integrate as integrate

# from dustmaps.bayestar import BayestarQuery
# Gaia.login(user='ktirimba', password='Hardenberg_49')

Using matplotlib backend: TkAgg


In [2]:
# Gaia.login(user='ktirimba', password='Hardenberg_49')
#G108.47-02.81-result
files = glob.glob('/disks/strw3/tirimba/Master_project/*.vot')
#print (files)
#f2 = '/home/tirimba/Downloads/1543249687118O-result.vot'
f2 = fits.open('/disks/strw3/tirimba/hrdiag_1m.fits')

def standardize(r,d,par,pmx,pmy):
	#coord_dist=np.degrees(np.arccos(np.sin(np.radians(d))*np.sin(np.radians(dec_maser))+np.cos(np.radians(d))*np.cos(np.radians(dec_maser))*np.cos(np.radians(r-ra_maser))))
	#print (coord_dist)
	par=par-par_maser
	pmx=pmx-pmra_maser
	pmy=pmy-pmdec_maser
	scaled_dist=zscore(coord_dist)
	scaled_par=zscore(par)
	scaled_pmx=zscore(pmx)
	scaled_pmy=zscore(pmy)
	#l=list(map(distance.euclidean,scaled_r,scaled_d,scaled_par,scaled_pmx,scaled_pmy))
	return np.sqrt(coord_dist**2 + scaled_par**2 + scaled_pmx**2+ scaled_pmy**2)
	
f='/disks/strw3/tirimba/Master_project/G111.23-01.23-result.vot'#, '/disks/strw3/tirimba/Master_project/G108.59+00.49', '/disks/strw3/tirimba/Master_project/G108.42+00.89-result.vot']


# table2 = parse_single_table(f2)
# print (table2)
# gaia_g= table2.array['g_rp']
# gaia_bprp=table2.array['bp_rp']
# gaia_par=table2.array['parallax']
# plt.plot(gaia_bprp,gaia_g+5 - 5*np.log10(1/(gaia_par/1e3)),'.',c='k')
# plt.show()


IOError: [Errno 2] No such file or directory: '/disks/strw3/tirimba/hrdiag_1m.fits'

In [ ]:
table=parse_single_table(f)

In [92]:
maser_name= f.split('-r')[0]
maser_name=maser_name.split('/')[-1]
#print (maser_name)
data= table.array
#print (np.shape(data))
#print (len(data))
#print (maser_name,len(data))
maser_list = pd.read_csv('masers.dat', sep=' ', header=0)
pos= np.where(maser_list['name']==maser_name)[0][0]
#print (pos)
pmra_maser= maser_list['pmra'][pos]
ra_maser= maser_list['ra'][pos]
#print (ra_maser)
pmdec_maser = maser_list['pmdec'][pos]
dec_maser=maser_list['dec'][pos]
par_maser = maser_list['parallax'][pos]
coords = str(maser_list['ra'][pos]) + str(maser_list['dec'][pos])
ra_maser=SkyCoord(coords, unit=(u.hourangle, u.degree)).ra.degree
dec_maser=SkyCoord(coords, unit=(u.hourangle, u.degree)).dec.degree
# print (ra_maser)
# print (dec_maser)
# print (pmra_maser)
# print (pmdec_maser)
#print (pmra_maser,pmdec_maser)
bprp=data['bp_rp']
abs_g= data['phot_g_mean_mag']+5. + 5.* np.log10((np.array(data['parallax'])/1e3))
# J_mag = data['j_m']
# K_mag= data['ks_m']
pmra= data['pmra']
pmdec = data['pmdec']
ra= data['ra']
dec=data['dec']
parallax=data['parallax']
#print ((standardize(ra,dec,parallax,pmra,pmdec)))




gaia_bprp = f2[1].data['bp_rp']
gaia_g = f2[1].data['mg']

xyrange = [[-1, 7], [-6, 16]]  # data range
bins = [300, 300]  # number of bins
thresh = 1  # density threshold

xdat = gaia_bprp
ydat = gaia_g

# histogram the data
hh, locx, locy = scipy.histogram2d(xdat, ydat, range=xyrange, bins=bins)
#print (coords)
# k="SELECT bp_rp,g_rp FROM gaiadr2.gaia_source"
# job = Gaia.launch_job(k)
# r = job.get_results()
# print (len(r))

# with open("num_associations.txt", "a") as myfile:
#     myfile.write( maser_name +': '+ str(len(data)) + '\n')
#     myfile.close()
radial_distance = (1/par_maser) * 1000
radius = (206265.0* (125.) /radial_distance) *(1/3600.)
near_points=np.degrees(np.arccos(np.sin(np.radians(dec))*np.sin(np.radians(dec_maser))
        +np.cos(np.radians(dec))*np.cos(np.radians(dec_maser))*np.cos(np.radians(ra-ra_maser))))<0.25*radius
#print ('distance in degrees',near_points)


# plt.scatter(pmra_maser,pmdec_maser,s=15,c='r')
# plt.scatter(pmra[(pmra>-3.12545) & (pmra<-3.0802)& (pmdec>-2.4289) & (pmdec<-2.3555)],pmdec[(pmra>-3.12545) & (pmra<-3.0802)&(pmdec>-2.4289) & (pmdec<-2.3555)],s=1,c='r')
# plt.scatter(pmra[(pmra > -3.379) & (pmra < -3.33) & (pmdec > -2.507) & (pmdec < -2.42)],pmdec[(pmra > -3.379) & (pmra < -3.33) & (pmdec > -2.507) & (pmdec < -2.42)], s=1, c='r')
# plt.scatter(-2.937553780176354,-2.7821059330868376,s=10,c='b')
# plt.scatter(-3.028231815513875, -2.648484169167197, s=10, c='y')
# plt.scatter(-3.27898034643, -2.691365382515417, s=10, c='m')
# plt.scatter(-3.202,-2.83315, s=10, c='g')
# plt.xlabel('$\mu _x$')
# plt.ylabel('$\mu _y$')
# plt.show()
#
# plt.scatter(ra,dec,s=1,c='k')
# plt.scatter(ra[(pmra>-3.12545) & (pmra<-3.0802)& (pmdec>-2.4289) & (pmdec<-2.3555)], dec[(pmra>-3.12545) & (pmra<-3.0802)& (pmdec>-2.4289) & (pmdec<-2.3555)], s=5, c='r')
# plt.scatter(ra[(pmra > -3.379) & (pmra < -3.33) & (pmdec > -2.507) & (pmdec < -2.42)],dec[(pmra > -3.379) & (pmra < -3.33) & (pmdec > -2.507) & (pmdec < -2.42)], s=5, c='r')
# plt.scatter(ra_maser,dec_maser,s=15,c='r')
# plt.scatter(345.3321585449551,56.994693451096, s=10, c='b')
# plt.scatter(345.1659552885973,56.8124282031640, s=10, c='y')
# plt.scatter(345.48669404800705,57.44909756778122, s=10, c='m')
# plt.scatter(346.4424512109195, 56.52536241639638, s=10, c='g')
# plt.xlabel('ra($^o$)')
# plt.ylabel('dec($^o$)')
# plt.show()

#a,b,c,d,e=standardize(ra[near_points],dec[near_points],parallax[near_points],pmra[near_points],pmdec[near_points])
#plt.plot(a,b,'.')
#plt.show()
distance = standardize(ra[near_points],dec[near_points],parallax[near_points],pmra[near_points],pmdec[near_points])

###histogram of the distances
#plt.hist(distance,bins=20)
#plt.title('d value')
#plt.ylabel('N')
#plt.show()


fig,axarr= plt.subplots(3,2,figsize=(8,8))

axarr[0,0].scatter(ra,dec,s=1,c='k',label='field')
axarr[0,0].scatter(ra[near_points],dec[near_points],s=2,c='c',label='nearby field')
axarr[0,0].scatter(ra_maser,dec_maser,c='g',s=10)
axarr[0,0].set_xlabel('RA')
axarr[0,0].set_ylabel('DEC')
axarr[0,0].legend(loc='upper right',fontsize=8)


###histogram of the distances
#axarr[0,1].hist(distance,bins=20)
#axarr[0,1].set_title('d value')
#axarr[0,1].set_ylabel('N')

print ('distance:',np.sort(distance)[:10])

##positions of nearest points in red
axarr[0,1].scatter(ra[near_points],dec[near_points],c='c',s=1,label='nearby field')
axarr[0,1].plot(ra[near_points][distance<1.0],dec[near_points][distance<1.0],'.',c='r',label='cluster?')
axarr[0,1].scatter(ra_maser,dec_maser,c='g',s=10,label='maser')
axarr[0,1].set_xlabel('RA')
axarr[0,1].set_ylabel('DEC')
axarr[0,1].legend(loc='upper right',fontsize=8)

#### proper motions
axarr[1,0].scatter(pmra,pmdec,c='k',s=1, label='field')
axarr[1,0].scatter(pmra[near_points],pmdec[near_points],c='c',s=1, label='nearby field')
axarr[1,0].plot(pmra[near_points][distance<1.0],pmdec[near_points][distance<1.0],'.',c='r',label='cluster?')
axarr[1,0].scatter(pmra_maser,pmdec_maser,c='g',s=30,label='maser')
axarr[1,0].set_xlabel('$\mu _x$')
axarr[1,0].set_ylabel('$\mu _y$')
axarr[1,0].set_xlim(-10,10)
axarr[1,0].set_ylim(-10,10)
axarr[1,0].legend(loc='upper right',fontsize=8)

#####CMD
axarr[1,1].scatter(bprp,abs_g,c='k',s=1,label='field')
axarr[1,1].scatter(bprp[near_points][distance<1.0],abs_g[near_points][distance<1.0],c='r',s=3,label='cluster?')
axarr[1,1].set_xlabel('Bp-Rp')
axarr[1,1].set_ylabel('G')
axarr[1,1].invert_yaxis()
axarr[1,1].legend(loc='upper right',fontsize=8)
#plt.savefig('/disks/strw5/tirimba/cluster'+maser_name+'.pdf')
x=ra[near_points][distance<1.0]
y=dec[near_points][distance<1.0]
ag=abs_g[near_points][distance<1.0]
x2=x[ag<1.5]
y2=y[ag<1.5]
# print ('mag',ag[ag<1.5])
# print ('coords',x2,y2)

n, bins, patches = axarr[2,0].hist(pmra[near_points][distance<1.0],20)
#ax1[0,0].plot(,bins=20)
axarr[2,0].set_title('$\mu _x$, $\sigma$='+str(np.std(pmra[near_points][distance<1.0])))
axarr[2,0].set_ylabel('N')
axarr[2,0].set_xlabel('$\mu _x$')

n2, bins2, patches2 = axarr[2,1].hist(distance[distance<1.0],30)
axarr[2,1].set_title('mean_d_value ='+str(np.mean(distance)))
axarr[2,1].set_ylabel('N')
axarr[2,1].set_xlabel('$d$')
axarr[2,1].set_xlim(0,1)
plt.savefig('/disks/strw5/tirimba/maser_d_val.pdf')
plt.show()

distance: [0.27121475420391683 0.28731731376041447 0.3538262546397256
 0.40157404458608986 0.43853385772592285 0.4995675244381838
 0.5182358693511133 0.5188634768808813 0.5200340747643377
 0.5348573133062224]


In [19]:
np.sort(distance)

masked_array(data=[0.24415512158791589, 0.2743724044740749,
                   0.40324855729480547, ..., 9.04834522356876,
                   13.529845369595686, 15.766669450054144],
             mask=[False, False, False, ..., False, False, False],
       fill_value=1e+20)

In [21]:
plt.hist(distance,bins=100)
plt.show()

In [ ]:
########### actual cluster



In [3]:
def standardize(l_s,b_s,par,par_error,pmx,pmy):
#def standardize(l_s,b_s,par,pmx,pmy):
    #position = np.random.randint(len(r))
    #print ('position',position)
    #coord_dist=np.degrees(np.arccos(np.sin(np.radians(d))*np.sin(np.radians(dec_center))+np.cos(np.radians(d))*np.cos(np.radians(dec_center))*np.cos(np.radians(r-ra_center))))
    l=l_s-l_center
    b=b_s-b_center
    par=par#-par_center
    pmx=pmx#-pmra_center
    pmy=pmy#-pmdec_center
#     scaled_dist=zscore(coord_dist)
    scaled_l=zscore(l)
    scaled_b=zscore(b)
    scaled_par=zscore(par)
#     plt.scatter(np.arange(len(scaled_par)),par)
#     plt.show()
    scaled_pmx=zscore(pmx)
    scaled_pmy=zscore(pmy)
    #return scaled_dist,scaled_par,scaled_pmx,scaled_pmy
    #l=list(map(distance.euclidean,scaled_r,scaled_d,scaled_par,scaled_pmx,scaled_pmy))
    #return np.sqrt(coord_dist**2+scaled_par**2 + scaled_pmx**2+ scaled_pmy**2)
    #return np.sqrt(scaled_par**2 + scaled_pmx**2+ scaled_pmy**2)
    return np.sqrt(scaled_l**2+scaled_b**2 + scaled_par**2 + scaled_pmx**2+ scaled_pmy**2)
def standardize2(pmx,pmy):
    pmx=pmx#-pmra_center
    pmy=pmy#-pmdec_center
    scaled_pmx=zscore(pmx)
    scaled_pmy=zscore(pmy)
    return scaled_pmx,scaled_pmy#np.sqrt(scaled_pmx**2+ scaled_pmy**2)


#print table
# print (table2)
# gaia_g= table2.array['g_rp']
# gaia_bprp=table2.array['bp_rp']
# gaia_par=table2.array['parallax']
# plt.plot(gaia_bprp,gaia_g+5 - 5*np.log10(1/(gaia_par/1e3)),'.',c='k')
# plt.show()
#maser_name= f.split('-r')[0]


In [4]:
def hdbscan_method(pmra,pmdec,parallax,l,b):

    
    X=np.column_stack((pmra,pmdec,parallax,l,b))
    X3 = np.column_stack((ra,dec))
    X2=StandardScaler().fit_transform(X)
    
    clusterer=hdbscan.HDBSCAN(min_cluster_size=5, gen_min_span_tree=True)
    clusterer.fit(X2)

    color_palette = sns.color_palette()
    try:
        cluster_colors = [color_palette[x] if x >= 0
                          else (0., 0., 0.)
                          for x in clusterer.labels_]
        cluster_member_colors = [sns.desaturate(x, p) for x, p in
                                 zip(cluster_colors, clusterer.probabilities_)]
        print clusterer.probabilities_

#         #plt.scatter(X[:,0].T, X[:,1].T, c=cluster_colors, **plot_kwds)
#         plt.scatter(X[:,0].T,X[:,1].T,facecolors=cluster_colors, **plot_kwds)
#         plt.errorbar(pmra_maser,pmdec_maser,xerr =pmra_maser_error,yerr=pmra_maser_error)
#         plt.xlim(-2+pmra_maser,2+pmra_maser)
#         plt.ylim(-2+pmdec_maser,2+pmdec_maser)
#         plt.show()
#         #plt.savefig(maser_name+'hdbscanclusters2'+'.pdf')
#         plt.close()
        
        
    except IndexError:
          cluster_colors = [(0., 0., 0.) for x in clusterer.labels_]
          
    return X3[:,0].T,X3[:,1].T

In [75]:
#match_list = glob.glob('*gaia_matches.vot')
match_list = glob.glob('*gaia_matches.vot')
pmra_cls=[]
member_cls=[]
ra_cls=[]
dec_cls=[]
pmdec_cls=[]
parallax_cls = []
parallax_err_cls = []
mag_g_cls=[]
bprp_cls=[]
for cl in match_list:
# cl = 'ngc_6231_gaia_matches.vot'
# #cl = 'ngc_6618_gaia_matches.vot'
    tabcl=parse_single_table(cl)
    pmra_cl=tabcl.array['pmra']
    print len(pmra_cl)
    ra_cls.append(tabcl.array['ra'])#[~np.isnan(pmra_cl)])
    member_cls.append(tabcl.array['source_id'])#[~np.isnan(pmra_cl)])
    dec_cls.append(tabcl.array['dec'])#[~np.isnan(pmra_cl)])
    pmdec_cls.append(tabcl.array['pmdec'])#[~np.isnan(pmra_cl)])
    parallax_cls.append(tabcl.array['parallax'])#[~np.isnan(pmra_cl)])
    #print (parallax_cl)
    parallax_err_cls.append(tabcl.array['parallax_error'])#[~np.isnan(pmra_cl)])
    mag_g_cls.append(tabcl.array['phot_g_mean_mag'])
    bprp_cls.append(tabcl.array['bp_rp'])
    #poe_cl=tabcl.array['parallax_over_error'][~npisnan(pmra_cl)]
    pmra_cls .append(pmra_cl)#[~np.isnan(pmra_cl)])
#print (ra_cl)
# import csv
# #for i in np.arange(len(ra_cl)):
# rows = zip(ra_cl,dec_cl)
plt.hist(parallax_cls,bins=20)#np.isnan(pmra_cl)
plt.show()
# with open('ngc_6618.csv', "w") as f:
#     writer = csv.writer(f)
#     for row in rows:
#         writer.writerow(row)

1890


In [5]:
# l_fields=['ngc_2439-result2.vot']
l_fields=['ngc_6611-result.vot','ascc_65-result.vot','ngc_6231-result.vot',
          'ngc_2439-result.vot','ngc_188-result.vot','ngc_6604-result.vot']

l_tables=[]
for f in l_fields:
    l_tables.append(parse_single_table(f))

In [6]:
l_clusters=['2m_2886_NGC_6611_2.dat','2m_1909_ASCC_65.dat','2m_2481_NGC_6231.dat','2m_1306_NGC_2439.dat',
            '2m_0074_NGC_188.dat','2m_2878_NGC_6604.dat']
ra_cen=[274.7000000,167.7750000,253.5583333 ,115.2000000,11.8500000,274.5083333]   
dec_cen=[-13.8066667,-61.1200000, -41.8188889,-31.7008333,85.2508333,-12.2341667]
l_cen=[16.954,291.1159,343.4769285,246.442,122.8415366,18.2513513]
b_cen= [0.793,-0.5714, 1.1783031,4.466,22.3797932,1.7019013]
pmra_cen=[0.62,-6.89 , -0.38,-2.22,0.15, -0.22] 
pmdec_cen=[-0.32,1.20,-1.35,3.13,-1.43,-2.70]
par_cen=[0.6,0.52,0.72,0.77,0.5,0.53]


In [21]:
metallicity,JMAG,KSMAG,logage=np.loadtxt('Av2.dat',usecols=(0,-3,-1,2),unpack=True)

In [22]:
for n,t in enumerate(l_tables):
    fraction=[]
    for j in np.arange(0.05,0.3,.01):
        l_center = l_cen[n]
        b_center = b_cen[n]
        pmra_center = pmra_cen[n]
        pmdec_center = pmdec_cen[n]
        par_center = par_cen[n]
        data = t.array
        '''
        #print (np.shape(data))
        #print (len(data))
        #print (maser_name,len(data))
        #maser_list = pd.read_csv('masers.dat', sep=' ', header=0)
        #pos= np.where(maser_list['name']==maser_name)[0][0]
        #pmra_maser= maser_list['pmra'][pos]
        #ra_maser= maser_list['ra'][pos]
        #print (ra_maser)
        #pmdec_maser = maser_list['pmdec'][pos]
        #dec_maser=maser_list['dec'][pos]
        coords = str(maser_list['ra'][pos]) + str(maser_list['dec'][pos])
        ra_maser=SkyCoord(coords, unit=(u.hourangle, u.degree)).ra.degree
        dec_maser=SkyCoord(coords, unit=(u.hourangle, u.degree)).dec.degree
        '''
        #print (ra_maser)
        #print (pmra_maser,pmdec_maser)
        # J_mag = data['j_m']
        # K_mag= data['ks_m']

        pmra=data['pmra']
        #print 'data length',len(pmra)
    #     print 'cluster length', len(ra_cls[n])
    #     print pmra
        source_id= data['source_id'][~np.isnan(pmra)]
        bprp=(data['bp_rp'])[~np.isnan(pmra)]
        g_mag = data['phot_g_mean_mag'][~np.isnan(pmra)]
        abs_g= (data['phot_g_mean_mag']+5. + 5.* np.log10((np.array(data['parallax'])/1e3)))[~np.isnan(pmra)]
        pmra =pmra[~np.isnan(pmra)]
        pmra_error=data['pmra_error'][~np.isnan(pmra)]
        pmdec = data['pmdec'][~np.isnan(pmra)]
        pmdec_error = data['pmdec_error'][~np.isnan(pmra)]
        ra= data['ra'][~np.isnan(pmra)]
    #     print ra
    #     print ra_cls
        dec=data['dec'][~np.isnan(pmra)]
        l_s=data['l'][~np.isnan(pmra)]
        b_s=data['b'][~np.isnan(pmra)]
        parallax=data['parallax'][~np.isnan(pmra)]
        par_error=data['parallax_error'][~np.isnan(pmra)]
    #     hdbra,hdbdec=hdbscan_method(pmra,pmdec,parallax,l_s,b_s)
    # #     ra_reduced=ra[standardize(l_s,b_s,parallax,par_error,pmra,pmdec)<0.8]
    # #     dec_reduced=dec[standardize(l_s,b_s,parallax,par_error,pmra,pmdec)<0.8]
    #     print 'frac',[n,cross_check(hdbra,hdbdec,l_clusters[n])]


    #     print len(ra_reduced)
    #     plt.hist(ra_reduced)
    #     plt.show()
    #     print (len(pmra))

    #         match_indices=np.sort(np.intersect1d(member_cls[n], source_id, assume_unique=False, return_indices=True)[1])
    #         print 'matching positions',len(match_indices)
    #     #     print len(ra_cl)
    #         print match_indices
    #     #     print len(match_indices)
    #         ra_c=ra_cls[n][match_indices]
    #         dec_c=dec_cls[n][match_indices]
    #         pmdec_c=pmdec_cls[n][match_indices]
    #         parallax_c=parallax_cls[n][match_indices]
    #         #print (parallax_cl)
    #         parallax_err_c=parallax_err_cls[n][match_indices]
    #         pmra_c =pmra_cls[n][match_indices]
    #         #print (len(pmra),len(pmdec))
    #         print len(pmra_c)

        #for j in np.arange(0.18,0.5,0.01):
        #for k in np.arange(10,20,1):
    #print ('ATTEMPT')
    #var_array=np.ones(len(pmra))*0.05
    #print (len(pmra))
    # np.random.seed(17)
    # pmx_rand=np.random.normal(-2.5,2.5,5000)
    # pmy_rand=np.random.normal(-1,1,5000)
    # pmx_rand=np.append(pmx_rand,pmra_c)
    # pmy_rand=np.append(pmy_rand,pmdec_c)

        X=np.column_stack((pmra,pmdec,parallax,l_s,b_s))
        #X=np.column_stack((pmra,pmdec))
        X2=StandardScaler().fit_transform(X)
        X3 = np.column_stack((bprp,g_mag,ra,dec))

        #X_2= np.column_stack((ra,dec,parallax,pmra))
        db=DBSCAN(eps=j,min_samples=5,  metric='euclidean').fit(X2)
        core_samples_mask = np.zeros_like(db.labels_, dtype=bool)
        core_samples_mask[db.core_sample_indices_] = True
        labels = db.labels_

        # Number of clusters in labels, ignoring noise if present.
        n_clusters_ = len(set(labels)) - (1 if -1 in labels else 0)
        n_noise_ = list(labels).count(-1)

        print('Estimated number of clusters: %d' % n_clusters_)
        print('Estimated number of noise points: %d' % n_noise_)
        # print("Homogeneity: %0.3f" % metrics.homogeneity_score(labels_true, labels))
        # print("Completeness: %0.3f" % metrics.completeness_score(labels_true, labels))
        # print("V-measure: %0.3f" % metrics.v_measure_score(labels_true, labels))
        # print("Adjusted Rand Index: %0.3f"
        #       % metrics.adjusted_rand_score(labels_true, labels))
        # print("Adjusted Mutual Information: %0.3f"
        #       % metrics.adjusted_mutual_info_score(labels_true, labels))
        # print("Silhouette Coefficient: %0.3f"
        #       % metrics.silhouette_score(X, labels))

        # #############################################################################
        # Plot result
        #plt.plot(pmra,pmdec,'.',c='0.5')
        #plt.plot(bprp_cls,mag_g_cls,'.',c='r')
        #plt.plot(pmra_cls,pmdec_cls,'.',c='r')
    #         import matplotlib.pyplot as plt
        #pl=np.array([,])
        # Black removed and is used for noise instead.
        unique_labels = set(labels)
        colors = [plt.cm.Spectral(each)
                  for each in np.linspace(0, 1, len(unique_labels))]
        #i=0
        for k, col in zip(unique_labels, colors):
            if k == -1:
                # Black used for noise.
                col = [0, 0, 0, 1]

            class_member_mask = (labels == k)


            xy = X[class_member_mask & core_samples_mask]
            xy3 = X3[class_member_mask & core_samples_mask]
    #             print np.shape(xy)
        # 		xy3 = X3[class_member_mask & ~core_samples_mask]
        # 		xy4 = X4[class_member_mask & ~core_samples_mask]
            #xy2=X2[class_member_mask]
            #plt.show()
            #print (xy3[:,0][xy2[:,1]<1.0],xy3[:,1][xy2[:,1]<1.0])
            #hh[hh < thresh] = np.nan  # fill the areas with low density by NaNs
            #

            #plt.imshow(np.sqrt(np.flipud(hh.T)), cmap='hot', extent=np.array(xyrange).flatten(), interpolation='none',
                       #origin='upper', aspect='auto')
            #plt.scatter(xy[:,0],xy[:,1],c='b',s=1)
            #plt.plot(,s=1,c='b')
            #plt.xlabel('BP_RP,')
            #plt.gca().invert_yaxis()
            #plt.savefig('/home/tirimba/downloads/'+'plot'+str(i))
            #plt.show()
            #print (len(class_member_mask))
            if len(X[class_member_mask & core_samples_mask])>2:
                if len(xy[:,0])> 5:
                    fraction.append([n,cross_check(xy3[:,2],xy3[:,3],l_clusters[n])])
        
                    plt.plot(j,cross_check(xy3[:,2],xy3[:,3],l_clusters[n]),'.',c='k')
plt.title('DBSCAN Efficiency')
plt.xlabel('$\epsilon$')
plt.ylabel('N$_{68}$')
plt.show()
#                 pmeff=len((set(pmra_c).intersection(xy[:,0])))/(len(pmra_c))

#                 yieldpm=len((set(pmra_c).intersection(xy[:,0])))/ (len(xy[:,0]))
            #print len(set(pmra_c).intersection(xy[:,0]))
            #print 'cluster length',len(xy[:,0])
#                 print ('effectiveness',pmeff)
#                 print ('yield',yieldpm)
#                 ra_cand.append(xy3[:,2])
#                 dec_cand.append(xy3[:,3])

#                 cmap = plt.cm.rainbow
#                 plt.plot(j,yieldpm,'.',c='r')
#                 plt.plot(j,pmeff,'.',c='k')
                #plt.plot(ra_c, dec_c, '.', c='b')
#                     plt.plot(xy[:,0], xy[:,1], '.', c='b',alpha=0.3,markersize=20)
    #                 plt.plot(xy[:, 0], xy[:, 1], 'o', markerfacecolor=tuple(col),
    #                 markeredgecolor='k', markersize=6)
    #                 plt.plot(xy3[:, 0], xy3[:, 1]-5*np.log10(1e3/(1/3.9))+5, 'o', markerfacecolor=tuple(col),

    #                 markeredgecolor='k', markersize=6)
    #                 plt.plot(JMAG[logage==6.0]-KSMAG[logage==6.0],KSMAG[logage==6.0],c='b',label='1 Myr')
    #                 plt.plot(JMAG[logage==7.04139]-KSMAG[logage==7.04139],KSMAG[logage==7.04139],c='r',label='10 Myr')
    #                 plt.plot(JMAG[logage==7.95904]-KSMAG[logage==7.95904],KSMAG[logage==7.95904],c='g',label='100 Myr')
                    #plt.xlabel('distance value')
                    #plt.ylabel('% of cluster found')
    #     plt.xlim(-1,2)
    #     plt.ylim(-8,7)
#         plt.legend()
#         plt.gca().invert_yaxis()
# plt.show()
# plt.scatter(ra,dec,s=6,c='k')
# plt.scatter(ra_cl,dec_cl,s=1,c='c')
# plt.show()

# ra_center=115.2000000     
# dec_center=-31.7008333
# l_center=246.442
# b_center= 4.466
# pmra_center=-2.22 
# pmdec_center=3.13
# par_center=0.769

# ra_center=274.5706667    
# dec_center=-16.9015278
# pmra_center=0.68
# pmdec_center=-1.42 
# par_center=0.505

# ra=np.append(ra,ra_cl)
# dec=np.append(dec,dec_cl)
# pmra=np.append(pmra,pmra_cl)
# pmdec=np.append(pmdec,pmdec_cl)
# parallax=np.append(parallax,parallax_cl)
# #print len(parallax[parallax<0.])
# par_error=np.append(par_error,parallax_err_cl)

# distance = standardize(ra,dec,parallax,pmra,pmdec,par_error)
# distance2 = standardize2(pmra,pmdec)

# plt.hist(pmra_cl,bins=40)
# plt.show()
#near_points=np.degrees(np.arccos(np.sin(np.radians(dec))*np.sin(np.radians(dec_center))+np.cos(np.radians(dec))*np.cos(np.radians(dec_center))*np.cos(np.radians(ra-ra_center))))<(0.25*2.0)
# plt.plot(l_s,b_s,'.',c='k')
# plt.plot(l_center,b_center,'.',c='r')
# plt.show()
# plt.plot(ra[distance<1.0],dec[distance<1.0],'.')
# print (len(l_s[distance<1.0]))
# plt.show()
# plt.plot(pmra,pmdec,'.',c='k')
# plt.plot(pmra[distance<1.0],pmdec[distance<1.0],'.')
# #print ('std:',np.std(pmra[distance<1.0]),np.std(pmdec[distance<1.0]))
# plt.show()
# plt.plot(bprp,g_mag,'.',c='k')
# plt.plot(bprp[distance<1.0],g_mag[distance<1.0],'.')
# plt.gca().invert_yaxis()
# plt.show()
#print (len(f[0][:]))

#print len(pmra_cl)
# plt.scatter(pmra,pmdec,s=3,c='k')
# plt.scatter(pmra_cl,pmdec_cl,s=1,c='c')
# plt.show()
#plt.scatter(pmra[distance2<0.2],pmdec[distance2<0.2],s=1,c='c')
#plt.scatter(pmra[distance<1.0],pmdec[distance<1.0],s=1,c='r')
#print len(pmra[distance2<0.5])
#print len(np.array((set(pmra_cl).intersection(pmra))[0]))
#print len((set(pmra_cl).intersection(pmra[distance2<0.5])))
# ind=np.arange(0.2,2.0,0.05)
# for j in ind:
#     pm_eff=len((set(ra_cl).intersection(ra[distance2<j])))/(len(ra_cl))
#     par_eff=len((set(ra_cl).intersection(ra[distance<j])))/(len(ra_cl))
#     print (pm_eff)
# #     print (pm_eff)
#     yield_pm=len((set(ra_cl).intersection(ra[distance2<j])))/ (len(ra[distance2<j]))
#     yield_par=len((set(ra_cl).intersection(ra[distance<j])))/ (len(ra[distance<j]))
# #     print (yiel)
#     #print (j, len((set(ra_cl).intersection(ra[distance<j]))))
# #     plt.scatter(j,pm_eff,c='k',s=1,label='pmra+pmdec')
# #     plt.scatter(j,par_eff,c='r',s=1,label='pmra+pmdec+parallax')
#     plt.scatter(j,yield_pm,c='k',s=1,label='pmra+pmdec')
#     plt.scatter(j,yield_par,c='r',s=1,label='pmra+pmdec+parallax')
# plt.title('yield pm vs pm&parallax')
# #plt.legend()
# plt.show()
#plt.hist(parallax[distance<0.5])
#print 'proper motion efficiency:', 1.*pm_eff/len(pmra[distance2<1.0])
#print 'parallax included efficiency:',1.*par_eff/len(pmra[distance<1.0])
#plt.show()
#pmra_cl=pmra_cl[(pmra_cl>-2.8) & (pmra_cl<-1.75)]
# for i in range(len(pmra_cl)):
#     print pmra_cl[i],
# plt.hist(pmra[distance<0.2],bins=20,label='cluster1?')
# plt.hist(pmra[distance2<0.5],bins=20,label='cluster2?')
# plt.hist(pmra_cl,bins=50,label='actual_cluster')
# plt.legend()
# plt.show()'''


Estimated number of clusters: 0
Estimated number of noise points: 8460
Estimated number of clusters: 0
Estimated number of noise points: 8460
Estimated number of clusters: 0
Estimated number of noise points: 8460
Estimated number of clusters: 0
Estimated number of noise points: 8460
Estimated number of clusters: 0
Estimated number of noise points: 8460
Estimated number of clusters: 1
Estimated number of noise points: 8455
Estimated number of clusters: 1
Estimated number of noise points: 8454
Estimated number of clusters: 4
Estimated number of noise points: 8431
Estimated number of clusters: 5
Estimated number of noise points: 8414


/Users/admin/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:18: RuntimeWarning: invalid value encountered in long_scalars


Estimated number of clusters: 5
Estimated number of noise points: 8397
Estimated number of clusters: 4
Estimated number of noise points: 8391
Estimated number of clusters: 3
Estimated number of noise points: 8380
Estimated number of clusters: 3
Estimated number of noise points: 8372
Estimated number of clusters: 3
Estimated number of noise points: 8365
Estimated number of clusters: 4
Estimated number of noise points: 8351
Estimated number of clusters: 5
Estimated number of noise points: 8339
Estimated number of clusters: 5
Estimated number of noise points: 8328
Estimated number of clusters: 6
Estimated number of noise points: 8309
Estimated number of clusters: 7
Estimated number of noise points: 8286
Estimated number of clusters: 11
Estimated number of noise points: 8258
Estimated number of clusters: 14
Estimated number of noise points: 8235
Estimated number of clusters: 15
Estimated number of noise points: 8211
Estimated number of clusters: 22
Estimated number of noise points: 8157
Es

Estimated number of clusters: 2
Estimated number of noise points: 2514
Estimated number of clusters: 0
Estimated number of noise points: 6319
Estimated number of clusters: 0
Estimated number of noise points: 6319
Estimated number of clusters: 0
Estimated number of noise points: 6319
Estimated number of clusters: 0
Estimated number of noise points: 6319
Estimated number of clusters: 1
Estimated number of noise points: 6314
Estimated number of clusters: 1
Estimated number of noise points: 6314
Estimated number of clusters: 3
Estimated number of noise points: 6304
Estimated number of clusters: 3
Estimated number of noise points: 6303
Estimated number of clusters: 2
Estimated number of noise points: 6303
Estimated number of clusters: 2
Estimated number of noise points: 6303
Estimated number of clusters: 2
Estimated number of noise points: 6303
Estimated number of clusters: 4
Estimated number of noise points: 6293
Estimated number of clusters: 5
Estimated number of noise points: 6284
Estima

In [229]:
# match_indices=np.sort(np.intersect1d(member_cl, source_id, assume_unique=False, return_indices=True)[1])
# print len(member_cl)
# print len(ra_cl)
# print match_indices
# print len(match_indices)
# ra_c=ra_cl[match_indices]
# dec_c=dec_cl[match_indices]
# pmdec_c=pmdec_cl[match_indices]
# parallax_c=parallax_cl[match_indices]
# #print (parallax_cl)
# parallax_err_c=parallax_err_cl[match_indices]
# pmra_c =pmra_cl[match_indices]
# print(set(ra_cl).intersection(ra))
# print np.intersect1d(dec_cl, dec, assume_unique=False, return_indices=True)
# print np.where(dec==-31.682110772051054)
# print np.where(ra==115.15774539126637)
#print dec[761]
#print dec_cl[45]



29
29
[ 1  2  4  5  6  7  8  9 10 11 12 13 14 17 18 19 20 21 22 23 24 25 26 28]
24


In [9]:
plt.hist(parallax,bins=20)
#plt.scatter(ra_c,dec_c,s=1,c='c')
plt.show()
print np.mean(parallax_c)
print np.mean(pmra_c)
print np.mean(pmdec_c)

NameError: name 'parallax' is not defined

In [103]:
# ra_center=11.8666667   
# dec_center=85.2550000
# pmra_center=-2.58
# pmdec_center=0.17
# par_center=0.5
for j in np.arange(0.05,0.5,0.01):
    #for k in np.arange(10,20,1):
#print ('ATTEMPT')
#var_array=np.ones(len(pmra))*0.05
#print (len(pmra))
# np.random.seed(17)
# pmx_rand=np.random.normal(-2.5,2.5,5000)
# pmy_rand=np.random.normal(-1,1,5000)
# pmx_rand=np.append(pmx_rand,pmra_c)
# pmy_rand=np.append(pmy_rand,pmdec_c)
        X=np.column_stack((pmra,pmdec))
        #X=np.column_stack((pmra,pmdec))
        X2=StandardScaler().fit_transform(X)

        #X_2= np.column_stack((ra,dec,parallax,pmra))
        db=DBSCAN(eps=0.15,min_samples=k,  metric='euclidean').fit(X)
        core_samples_mask = np.zeros_like(db.labels_, dtype=bool)
        core_samples_mask[db.core_sample_indices_] = True
        labels = db.labels_

        # Number of clusters in labels, ignoring noise if present.
        n_clusters_ = len(set(labels)) - (1 if -1 in labels else 0)
        n_noise_ = list(labels).count(-1)

        print('Estimated number of clusters: %d' % n_clusters_)
        print('Estimated number of noise points: %d' % n_noise_)
        # print("Homogeneity: %0.3f" % metrics.homogeneity_score(labels_true, labels))
        # print("Completeness: %0.3f" % metrics.completeness_score(labels_true, labels))
        # print("V-measure: %0.3f" % metrics.v_measure_score(labels_true, labels))
        # print("Adjusted Rand Index: %0.3f"
        #       % metrics.adjusted_rand_score(labels_true, labels))
        # print("Adjusted Mutual Information: %0.3f"
        #       % metrics.adjusted_mutual_info_score(labels_true, labels))
        # print("Silhouette Coefficient: %0.3f"
        #       % metrics.silhouette_score(X, labels))

        # #############################################################################
        # Plot result
        import matplotlib.pyplot as plt
        #pl=np.array([,])
        # Black removed and is used for noise instead.
        unique_labels = set(labels)
        colors = [plt.cm.Spectral(each)
                  for each in np.linspace(0, 1, len(unique_labels))]
        #i=0
        for k, col in zip(unique_labels, colors):
            if k == -1:
                # Black used for noise.
                col = [0, 0, 0, 1]

            class_member_mask = (labels == k)


            xy = X[class_member_mask & ~core_samples_mask]
            print np.shape(xy)
        # 		xy3 = X3[class_member_mask & ~core_samples_mask]
        # 		xy4 = X4[class_member_mask & ~core_samples_mask]
            #xy2=X2[class_member_mask]
            #plt.show()
            #print (xy3[:,0][xy2[:,1]<1.0],xy3[:,1][xy2[:,1]<1.0])
            #hh[hh < thresh] = np.nan  # fill the areas with low density by NaNs
            #

            #plt.imshow(np.sqrt(np.flipud(hh.T)), cmap='hot', extent=np.array(xyrange).flatten(), interpolation='none',
                       #origin='upper', aspect='auto')
            #plt.scatter(xy[:,0],xy[:,1],c='b',s=1)
            #plt.plot(,s=1,c='b')
            #plt.xlabel('BP_RP,')
            #plt.gca().invert_yaxis()
            #plt.savefig('/home/tirimba/downloads/'+'plot'+str(i))
            #plt.show()
            #print (len(class_member_mask))
            if len(X[class_member_mask & core_samples_mask])>5:
#                 pmeff=len((set(pmra_c).intersection(xy[:,0])))/(len(pmra_c))

#                 yieldpm=len((set(pmra_c).intersection(xy[:,0])))/ (len(xy[:,0]))
#                 print len(set(pmra_c).intersection(xy[:,0]))
#                 print ('effectiveness',pmeff)
                print ('yield',yieldpm)
                
                cross_check(xy3[:,])
#                 plt.scatter(j,yieldpm,s=1,c='r')
#                 plt.show()
        #         plt.scatter(bprp,abs_g,c='k',s=1)
        #         plt.scatter(xy[:,0],xy[:,1],c='r',s=3)
        #         plt.xlabel ('Bp-Rp')
        #         plt.ylabel('G')
        #         plt.gca().invert_yaxis()
        #         plt.show()
        #         plt.scatter(ra[near_points],dec[near_points],c='k',s=1,label='entire field')
        #         plt.scatter(xy3[:,0],xy3[:,1],c='r',s=3,label='nearby region')
        #         plt.scatter(ra_maser,dec_maser,c='c',s=10,label='maser')
        #         plt.xlabel('RA')
        #         plt.ylabel('DEC')
        #         plt.legend()
        #         plt.show()
        ##########################################################
        #         plt.scatter(pmra,pmdec,c='k',s=1)
        #         plt.scatter(pmra_c,pmdec_c,c='c',s=10)
        #         plt.scatter(xy[:,0],xy[:,1],c='r',s=1)
        #         plt.show()
        #             print ('found points:',len(xy[:,0]))
        #             #plt.scatter(pmra_maser,pmdec_maser,c='c',s=10,label='maser')
        #             plt.xlabel('$\mu _x$')
        #             plt.ylabel('$\mu _y$')
        #             #plt.legend()
        #             plt.show()

            #i+=1
            #print (np.shape(xy2))
            #plt.plot(xy2[:, 0], xy2[:, 1],'.')

            # xy = X[class_member_mask & ~core_samples_mask]
            # plt.plot(xy[:, 0], xy[:, 1], 'o', markerfacecolor=tuple(col),
            #          markeredgecolor='k', markersize=6,alpha=0.4)

        #plt.title('Estimated number of clusters: %d' % n_clusters_)

        #####################
        #X = np.column_stack((pmra,pmdec))
        #kmeans = KMeans(n_clusters=5)
        #kmeans.fit(X)
        #y_kmeans = kmeans.predict(X)
        #centers = kmeans.cluster_centers_
        # plt.scatter(pmra, pmdec, s=1, c='k')
        # plt.scatter(centers[:, 0], centers[:, 1], c='b', s=10, alpha=0.5)
        # plt.show()



Estimated number of clusters: 1961
Estimated number of noise points: 0
(0, 2)
(0, 2)


ZeroDivisionError: division by zero

In [87]:

######################################################################NOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOO


def standardize(r,d,par,pmx,pmy):
	#position = np.random.randint(len(r))
	#print ('position',position)
	coord_dist=np.degrees(np.arccos(np.sin(np.radians(d))*np.sin(np.radians(dec[position]))+np.cos(np.radians(d))*np.cos(np.radians(dec[position]))*np.cos(np.radians(r-ra[position]))))
	par=par-parallax[position]
	pmx=pmx-pmra[position]
	pmy=pmy-pmdec[position]
	scaled_dist=zscore(coord_dist)
	scaled_par=zscore(par)
	scaled_pmx=zscore(pmx)
	scaled_pmy=zscore(pmy)
	#l=list(map(distance.euclidean,scaled_r,scaled_d,scaled_par,scaled_pmx,scaled_pmy))
	return np.sqrt(coord_dist**2 + scaled_par**2 + scaled_pmx**2+ scaled_pmy**2)
	

In [ ]:
f='/disks/strw3/tirimba/Master_project/G188.94+00.88-result.vot'#, '/disks/strw3/tirimba/Master_project/G108.59+00.49', '/disks/strw3/tirimba/Master_project/G108.42+00.89-result.vot']

table=parse_single_table(f)

In [88]:
maser_name= f.split('-r')[0]
maser_name=maser_name.split('/')[-1]
print (maser_name)
data= table.array
#print (np.shape(data))
#print (len(data))
#print (maser_name,len(data))
maser_list = pd.read_csv('masers.dat', sep=' ', header=0)
pos= np.where(maser_list['name']==maser_name)[0][0]
#print (pos)
pmra_maser= maser_list['pmra'][pos]
ra_maser= maser_list['ra'][pos]
#print (ra_maser)
pmdec_maser = maser_list['pmdec'][pos]
dec_maser=maser_list['dec'][pos]
par_maser = maser_list['parallax'][pos]
coords = str(maser_list['ra'][pos]) + str(maser_list['dec'][pos])
ra_maser=SkyCoord(coords, unit=(u.hourangle, u.degree)).ra.degree
dec_maser=SkyCoord(coords, unit=(u.hourangle, u.degree)).dec.degree
print (ra_maser)
print (dec_maser)
print (pmra_maser)
print (pmdec_maser)
#print (pmra_maser,pmdec_maser)
bprp=data['bp_rp']
abs_g= data['phot_g_mean_mag']+5 + 5* np.log10((np.array(data['parallax'])/1e3))
# J_mag = data['j_m']
# K_mag= data['ks_m']
pmra= data['pmra']
pmdec = data['pmdec']
ra= data['ra']
dec=data['dec']
parallax=data['parallax']
#print ((standardize(ra,dec,parallax,pmra,pmdec)))




gaia_bprp = f2[1].data['bp_rp']
gaia_g = f2[1].data['mg']

xyrange = [[-1, 7], [-6, 16]]  # data range
bins = [300, 300]  # number of bins
thresh = 1  # density threshold

xdat = gaia_bprp
ydat = gaia_g

# histogram the data
hh, locx, locy = scipy.histogram2d(xdat, ydat, range=xyrange, bins=bins)
#print (coords)
# k="SELECT bp_rp,g_rp FROM gaiadr2.gaia_source"
# job = Gaia.launch_job(k)
# r = job.get_results()
# print (len(r))

# with open("num_associations.txt", "a") as myfile:
#     myfile.write( maser_name +': '+ str(len(data)) + '\n')
#     myfile.close()
radial_distance = (1/par_maser) * 1000
radius = (206265.0* (125.) /radial_distance) *(1/3600.)
for i in np.arange(10):
    position = np.random.randint(0,len(ra),size=1)[0]
    print (position)

    near_points=np.degrees(np.arccos(np.sin(np.radians(dec))*np.sin(np.radians(dec[position]))+np.cos(np.radians(dec))*np.cos(np.radians(dec[position]))*np.cos(np.radians(ra-ra[position]))))<0.25*2.#radius
    #near_points=np.degrees(np.arccos(np.sin(np.radians(dec))*np.sin(np.radians(dec_maser))+np.cos(np.radians(dec))*np.cos(np.radians(dec_maser))*np.cos(np.radians(ra-ra_maser))))<0.25*radius
    print ('distance in degrees',near_points)


    # plt.scatter(pmra_maser,pmdec_maser,s=15,c='r')
    # plt.scatter(pmra[(pmra>-3.12545) & (pmra<-3.0802)& (pmdec>-2.4289) & (pmdec<-2.3555)],pmdec[(pmra>-3.12545) & (pmra<-3.0802)&(pmdec>-2.4289) & (pmdec<-2.3555)],s=1,c='r')
    # plt.scatter(pmra[(pmra > -3.379) & (pmra < -3.33) & (pmdec > -2.507) & (pmdec < -2.42)],pmdec[(pmra > -3.379) & (pmra < -3.33) & (pmdec > -2.507) & (pmdec < -2.42)], s=1, c='r')
    # plt.scatter(-2.937553780176354,-2.7821059330868376,s=10,c='b')
    # plt.scatter(-3.028231815513875, -2.648484169167197, s=10, c='y')
    # plt.scatter(-3.27898034643, -2.691365382515417, s=10, c='m')
    # plt.scatter(-3.202,-2.83315, s=10, c='g')
    # plt.xlabel('$\mu _x$')
    # plt.ylabel('$\mu _y$')
    # plt.show()
    #
    # plt.scatter(ra,dec,s=1,c='k')
    # plt.scatter(ra[(pmra>-3.12545) & (pmra<-3.0802)& (pmdec>-2.4289) & (pmdec<-2.3555)], dec[(pmra>-3.12545) & (pmra<-3.0802)& (pmdec>-2.4289) & (pmdec<-2.3555)], s=5, c='r')
    # plt.scatter(ra[(pmra > -3.379) & (pmra < -3.33) & (pmdec > -2.507) & (pmdec < -2.42)],dec[(pmra > -3.379) & (pmra < -3.33) & (pmdec > -2.507) & (pmdec < -2.42)], s=5, c='r')
    # plt.scatter(ra_maser,dec_maser,s=15,c='r')
    # plt.scatter(345.3321585449551,56.994693451096, s=10, c='b')
    # plt.scatter(345.1659552885973,56.8124282031640, s=10, c='y')
    # plt.scatter(345.48669404800705,57.44909756778122, s=10, c='m')
    # plt.scatter(346.4424512109195, 56.52536241639638, s=10, c='g')
    # plt.xlabel('ra($^o$)')
    # plt.ylabel('dec($^o$)')
    # plt.show()

    #a,b,c,d,e=standardize(ra[near_points],dec[near_points],parallax[near_points],pmra[near_points],pmdec[near_points])
    #plt.plot(a,b,'.')
    #plt.show()
    distance = standardize(ra[near_points],dec[near_points],parallax[near_points],pmra[near_points],pmdec[near_points])
    #plt.scatter(ra,dec,s=1,c='k',label='field')
    #plt.scatter(ra[near_points],dec[near_points],s=2,c='c',label='nearby field')
    #plt.scatter(ra[position],dec[position],c='g',s=30,label='maser')
    #plt.show()

    ###histogram of the distances

    fig,axarr= plt.subplots(3,2,figsize=(8,8))

    axarr[0,0].scatter(ra,dec,s=1,c='k',label='field')
    axarr[0,0].scatter(ra[near_points],dec[near_points],s=2,c='c',label='nearby field')
    axarr[0,0].scatter(ra[position],dec[position],c='g',s=10)
    axarr[0,0].set_xlabel('RA')
    axarr[0,0].set_ylabel('DEC')
    axarr[0,0].legend(loc='upper right',fontsize=8)



    print ('distances:',np.sort(distance)[:10])

    ##positions of nearest points in red
    axarr[0,1].scatter(ra[near_points],dec[near_points],c='c',s=1,label='nearby field')
    axarr[0,1].plot(ra[near_points][distance<1.0],dec[near_points][distance<1.0],'.',c='r',label='cluster?')
    axarr[0,1].scatter(ra[position],dec[position],c='g',s=10,label='central star')
    axarr[0,1].set_xlabel('RA')
    axarr[0,1].set_ylabel('DEC')
    axarr[0,1].legend(loc='upper right',fontsize=8)


    #### proper motions
    axarr[1,0].scatter(pmra,pmdec,c='k',s=1, label='field')
    axarr[1,0].scatter(pmra[near_points],pmdec[near_points],c='c',s=1, label='nearby field')
    axarr[1,0].plot(pmra[near_points][distance<1.0],pmdec[near_points][distance<1.0],'.',c='r',label='cluster?')
    axarr[1,0].scatter(pmra[position],pmdec[position],c='g',s=30,label='central star')
    axarr[1,0].set_xlabel('$\mu _x$')
    axarr[1,0].set_ylabel('$\mu _y$')
    axarr[1,0].legend(loc='upper right',fontsize=8)
    axarr[1,0].set_xlim(-10,10)
    axarr[1,0].set_ylim(-10,10)

    #####CMD
    axarr[1,1].scatter(bprp,abs_g,c='k',s=1,label='field')
    axarr[1,1].scatter(bprp[near_points][distance<1.0],abs_g[near_points][distance<1.0],c='r',s=3,label='cluster?')
    axarr[1,1].set_xlabel('Bp-Rp')
    axarr[1,1].set_ylabel('G')
    axarr[1,1].invert_yaxis()
    axarr[1,1].legend(loc='upper right',fontsize=8)

    n, bins, patches = axarr[2,0].hist(pmra[near_points][distance<1.0],20)
    #ax1[0,0].plot(,bins=20)
    axarr[2,0].set_title('$\mu _x$, $\sigma$='+str(np.std(pmra[near_points][distance<1.0])))
    axarr[2,0].set_ylabel('N')
    axarr[2,0].set_xlabel('$\mu _x$')

    n2, bins2, patches2 = axarr[2,1].hist(distance[distance<1.0],30)
    axarr[2,1].set_title('mean_d_value ='+str(np.mean(distance)))
    axarr[2,1].set_ylabel('N')
    axarr[2,1].set_xlabel('$d$')
    axarr[2,1].set_xlim(0,1)
    #plt.savefig('/disks/strw5/tirimba/cluster'+maser_name+'.pdf')
    #x=ra[near_points][distance<1.0]
    #y=dec[near_points][distance<1.0]
    #ag=abs_g[near_points][distance<1.0]
    #x2=x[ag<1.5]
    #y2=y[ag<1.5]
    #print ('mag',ag[ag<1.5])
    #print ('coords',x2,y2)
    #plt.show()
    plt.savefig('/disks/strw5/tirimba/field_dval_'+str(i)+'.pdf')

G111.23-01.23
349.33662
59.47971388888889
-4.37
-2.38
1892
distance in degrees [False False False ... False False False]
distances: [0.3352268648897422 0.33807421051423947 0.36281028583770353
 0.37761056337794763 0.39208216903921317 0.3974786699082403
 0.4109430865161832 0.41392351402212985 0.4175785692402729
 0.45463299548382186]
1269
distance in degrees [False False False ... False False False]
distances: [0.30192229436480056 0.37147949802834074 0.4028222145774506
 0.4062545939358519 0.443383483105663 0.4437001235964347
 0.4578249788507001 0.463332986474447 0.4815064389279537
 0.4926713566277262]
3367
distance in degrees [False False False ... False False False]
distances: [0.3407492856492306 0.4153215990749321 0.42427321862015466
 0.45301009831485883 0.4532929367120117 0.47304359644112876
 0.4846179631852068 0.5037460311419606 0.5197693791983257
 0.5310631632712335]
1725
distance in degrees [False False False ... False False False]
distances: [0.2986481081920197 0.349932640174494 0.

In [125]:
# for j in np.arange(0.02,0.1,0.01):
#     print ('ATTEMPT')
X = np.column_stack(distance2(pmra,pmdec))
print (np.shape(X))
db=DBSCAN(eps=0.05,min_samples=10,  metric='euclidean').fit(X)
core_samples_mask = np.zeros_like(db.labels_, dtype=bool)
core_samples_mask[db.core_sample_indices_] = True
labels = db.labels_

# Number of clusters in labels, ignoring noise if present.
n_clusters_ = len(set(labels)) - (1 if -1 in labels else 0)
n_noise_ = list(labels).count(-1)
print 
print('Estimated number of clusters: %d' % n_clusters_)
print('Estimated number of noise points: %d' % n_noise_)
# print("Homogeneity: %0.3f" % metrics.homogeneity_score(labels_true, labels))
# print("Completeness: %0.3f" % metrics.completeness_score(labels_true, labels))
# print("V-measure: %0.3f" % metrics.v_measure_score(labels_true, labels))
# print("Adjusted Rand Index: %0.3f"
#       % metrics.adjusted_rand_score(labels_true, labels))
# print("Adjusted Mutual Information: %0.3f"
#       % metrics.adjusted_mutual_info_score(labels_true, labels))
# print("Silhouette Coefficient: %0.3f"
#       % metrics.silhouette_score(X, labels))

# #############################################################################
# Plot result
import matplotlib.pyplot as plt
#pl=np.array([,])
# Black removed and is used for noise instead.
unique_labels = set(labels)
colors = [plt.cm.Spectral(each)
          for each in np.linspace(0, 1, len(unique_labels))]
#i=0
for k, col in zip(unique_labels, colors):
    if k == -1:
        # Black used for noise.
        col = [0, 0, 0, 1]

    class_member_mask = (labels == k)


    xy = X[class_member_mask & ~core_samples_mask]
# 		xy3 = X3[class_member_mask & ~core_samples_mask]
# 		xy4 = X4[class_member_mask & ~core_samples_mask]
    #xy2=X2[class_member_mask]
    #plt.show()
    #print (xy3[:,0][xy2[:,1]<1.0],xy3[:,1][xy2[:,1]<1.0])
    #hh[hh < thresh] = np.nan  # fill the areas with low density by NaNs
    #

    #plt.imshow(np.sqrt(np.flipud(hh.T)), cmap='hot', extent=np.array(xyrange).flatten(), interpolation='none',
               #origin='upper', aspect='auto')
    #plt.scatter(xy[:,0],xy[:,1],c='b',s=1)
    #plt.plot(,s=1,c='b')
    #plt.xlabel('BP_RP,')
    #plt.gca().invert_yaxis()
    #plt.savefig('/home/tirimba/downloads/'+'plot'+str(i))
    #plt.show()
    #print (len(class_member_mask))
    if len(X[class_member_mask & core_samples_mask])>5:
        pm_eff=len((set(pmra_cl).intersection(xy[:,0])))/(1.0*len(ra_cl))

#         plt.scatter(bprp,abs_g,c='k',s=1)
#         plt.scatter(xy[:,0],xy[:,1],c='r',s=3)
#         plt.xlabel ('Bp-Rp')
#         plt.ylabel('G')
#         plt.gca().invert_yaxis()
#         plt.show()
#         plt.scatter(ra[near_points],dec[near_points],c='k',s=1,label='entire field')
#         plt.scatter(xy3[:,0],xy3[:,1],c='r',s=3,label='nearby region')
#         plt.scatter(ra_maser,dec_maser,c='c',s=10,label='maser')
#         plt.xlabel('RA')
#         plt.ylabel('DEC')
#         plt.legend()
#         plt.show()

        #plt.scatter(pmra,pmdec,c='k',s=1)
        plt.scatter(pmra_cl,pmdec_cl,c='c',s=10)
        plt.scatter(xy[:,0],xy[:,1],c='r',s=1)
        plt.show()
        print ('found points:',len(xy[:,0]))
#             #plt.scatter(pmra_maser,pmdec_maser,c='c',s=10,label='maser')
#             plt.xlabel('$\mu _x$')
#             plt.ylabel('$\mu _y$')
#             #plt.legend()
#             plt.show()

    #i+=1
    #print (np.shape(xy2))
    #plt.plot(xy2[:, 0], xy2[:, 1],'.')

    # xy = X[class_member_mask & ~core_samples_mask]
    # plt.plot(xy[:, 0], xy[:, 1], 'o', markerfacecolor=tuple(col),
    #          markeredgecolor='k', markersize=6,alpha=0.4)

#plt.title('Estimated number of clusters: %d' % n_clusters_)

#####################
#X = np.column_stack((pmra,pmdec))
#kmeans = KMeans(n_clusters=5)
#kmeans.fit(X)
#y_kmeans = kmeans.predict(X)
#centers = kmeans.cluster_centers_
# plt.scatter(pmra, pmdec, s=1, c='k')
# plt.scatter(centers[:, 0], centers[:, 1], c='b', s=10, alpha=0.5)
# plt.show()




NameError: name 'par' is not defined

In [64]:
ngc_188_cl = pd.read_csv('ngc_6611.dat', delimiter = '|')#,skip_blank_lines=True)
#print ngc_188_cl.keys()
# print np.shape(ngc_188_cl)
prob_val=ngc_188_cl['prob. '].values
print prob_val
#y = prob_val.astype(np.float)

#print ngc_188_cl['prob. ']
#print len(ngc_188_cl['prob. '].values[3])
#ngc_188_cl = ngc_188_cl.loc[ngc_188_cl['prob. '] == '   100']
# #print ngc_188_cl['prob. ']
#ngc_188_cl = ngc_188_cl.loc[ngc_188_cl['typ'] == '*iC']
coord_vals= ngc_188_cl['       coord1 (ICRS,J2000/2000)        '].values
with open("ngc_6611.csv", "a") as myfile:
    myfile.write('ra|dec|pmra|pmra_error|pmdec|pmdec_error|parallax|parallax_error|\n')
    for i in coord_vals:
        try:
            coord = SkyCoord(i, unit=(u.hourangle, u.degree), frame='icrs')
            radius = u.Quantity(2.0, u.arcsec)
            j = Gaia.cone_search_async(coord, radius)
            r = j.get_results()
            print r
            myfile.write(str(r['ra'][0]) +'|'+ str(r['dec'][0]) +'|'+  str(r['pmra'][0])  +'|'+  str(r['pmra_error'][0]) +'|'+
                         str(r['pmdec'][0])+'|'+ str(r['pmdec_error'][0]) +'|'+ str(r['parallax'][0]) 
                         +'|'+ str(r['parallax_error'][0])+'\n')
        except IndexError:
            pass
    myfile.close()

#     for i in coord_vals:
#         print i
#         myfile.write(i + '\n')
#     myfile.close()
# print type(ngc_188_cl['prob. '].iloc[2])
# print len(ngc_188_cl['prob. '])

[ 75 100  75 ...   0   0   0]
Query finished.
         dist             solution_id     ... epoch_photometry_url
                                          ...                     
--------------------- ------------------- ... --------------------
7.519250120574091e-06 1635721458409799680 ...                   --
Query finished.
         dist             solution_id     ... epoch_photometry_url
                                          ...                     
--------------------- ------------------- ... --------------------
7.519250120574091e-06 1635721458409799680 ...                   --
Query finished.
         dist             solution_id     ... epoch_photometry_url
                                          ...                     
--------------------- ------------------- ... --------------------
1.918192282957428e-05 1635721458409799680 ...                   --
Query finished.
         dist             solution_id     ... epoch_photometry_url
                                   

Query finished.
         dist              solution_id     ... epoch_photometry_url
                                           ...                     
---------------------- ------------------- ... --------------------
1.4309470003683421e-05 1635721458409799680 ...                   --
Query finished.
         dist              solution_id     ... epoch_photometry_url
                                           ...                     
---------------------- ------------------- ... --------------------
1.4309470003683421e-05 1635721458409799680 ...                   --
Query finished.
         dist              solution_id     ... epoch_photometry_url
                                           ...                     
---------------------- ------------------- ... --------------------
1.4309470003683421e-05 1635721458409799680 ...                   --
Query finished.
         dist             solution_id     ... epoch_photometry_url
                                          ...        

Query finished.
         dist              solution_id     ... epoch_photometry_url
                                           ...                     
---------------------- ------------------- ... --------------------
1.7408629786660817e-05 1635721458409799680 ...                   --
Query finished.
         dist              solution_id     ... epoch_photometry_url
                                           ...                     
---------------------- ------------------- ... --------------------
1.7408629786660817e-05 1635721458409799680 ...                   --
Query finished.
         dist          ...
                       ...
---------------------- ...
1.4605592454421514e-05 ...
Query finished.
         dist          ...
                       ...
---------------------- ...
1.4605592454421514e-05 ...
Query finished.
         dist              solution_id     ... epoch_photometry_url
                                           ...                     
---------------------- -

Query finished.
        dist             solution_id     ... epoch_photometry_url
                                         ...                     
-------------------- ------------------- ... --------------------
8.78414520628598e-06 1635721458409799680 ...                   --
Query finished.
         dist              solution_id     ... epoch_photometry_url
                                           ...                     
---------------------- ------------------- ... --------------------
1.3471149778057768e-05 1635721458409799680 ...                   --
Query finished.
         dist              solution_id     ... epoch_photometry_url
                                           ...                     
---------------------- ------------------- ... --------------------
1.3471149778057768e-05 1635721458409799680 ...                   --
Query finished.
         dist              solution_id     ... epoch_photometry_url
                                           ...              

Query finished.
         dist              solution_id     ... epoch_photometry_url
                                           ...                     
---------------------- ------------------- ... --------------------
1.9535970965195302e-05 1635721458409799680 ...                   --
Query finished.
         dist              solution_id     ... epoch_photometry_url
                                           ...                     
---------------------- ------------------- ... --------------------
1.9535970965195302e-05 1635721458409799680 ...                   --
Query finished.
         dist              solution_id     ... epoch_photometry_url
                                           ...                     
---------------------- ------------------- ... --------------------
1.4099094456006129e-05 1635721458409799680 ...                   --
Query finished.
         dist              solution_id     ... epoch_photometry_url
                                           ...      

...

Query finished.
         dist             solution_id     ... epoch_photometry_url
                                          ...                     
--------------------- ------------------- ... --------------------
6.654423820249593e-06 1635721458409799680 ...                   --
Query finished.
dist solution_id designation ... datalink_url epoch_photometry_url
                             ...                                  
---- ----------- ----------- ... ------------ --------------------
Query finished.
         dist              solution_id     ... epoch_photometry_url
                                           ...                     
---------------------- ------------------- ... --------------------
0.00016350711457851287 1635721458409799680 ...                   --
Query finished.
dist solution_id designation ... datalink_url epoch_photometry_url
                             ...                                  
---- ----------- ----------- ... ------------ ---------------

Query finished.
dist solution_id designation ... datalink_url epoch_photometry_url
                             ...                                  
---- ----------- ----------- ... ------------ --------------------
Query finished.
         dist             solution_id     ... epoch_photometry_url
                                          ...                     
--------------------- ------------------- ... --------------------
0.0003811007870002523 1635721458409799680 ...                   --
Query finished.
         dist             solution_id     ... epoch_photometry_url
                                          ...                     
--------------------- ------------------- ... --------------------
7.427768840417694e-06 1635721458409799680 ...                   --
Query finished.
dist solution_id designation ... datalink_url epoch_photometry_url
                             ...                                  
---- ----------- ----------- ... ------------ -------------------

Query finished.
         dist              solution_id     ... epoch_photometry_url
                                           ...                     
---------------------- ------------------- ... --------------------
2.4933942936985315e-05 1635721458409799680 ...                   --
Query finished.
dist solution_id designation ... datalink_url epoch_photometry_url
                             ...                                  
---- ----------- ----------- ... ------------ --------------------
Query finished.
        dist             solution_id     ... epoch_photometry_url
                                         ...                     
-------------------- ------------------- ... --------------------
1.43592593586959e-05 1635721458409799680 ...                   --
Query finished.
dist solution_id designation ... datalink_url epoch_photometry_url
                             ...                                  
---- ----------- ----------- ... ------------ -------------------

Query finished.
         dist              solution_id     ... epoch_photometry_url
                                           ...                     
---------------------- ------------------- ... --------------------
2.3862641055408755e-05 1635721458409799680 ...                   --
 0.0005466188454622962 1635721458409799680 ...                   --
Query finished.
         dist              solution_id     ... epoch_photometry_url
                                           ...                     
---------------------- ------------------- ... --------------------
1.6639830974517686e-05 1635721458409799680 ...                   --
Query finished.
         dist              solution_id     ... epoch_photometry_url
                                           ...                     
---------------------- ------------------- ... --------------------
5.3202115845608934e-05 1635721458409799680 ...                   --
Query finished.
         dist              solution_id     ... epoch

Query finished.
         dist              solution_id     ... epoch_photometry_url
                                           ...                     
---------------------- ------------------- ... --------------------
2.3719142750158995e-05 1635721458409799680 ...                   --
 0.0001522830437322547 1635721458409799680 ...                   --
Query finished.
         dist              solution_id     ... epoch_photometry_url
                                           ...                     
---------------------- ------------------- ... --------------------
2.1760174412344798e-05 1635721458409799680 ...                   --
Query finished.
         dist              solution_id     ... epoch_photometry_url
                                           ...                     
---------------------- ------------------- ... --------------------
2.8316104492464606e-05 1635721458409799680 ...                   --
Query finished.
         dist             solution_id     ... epoch_

In [ ]:

coord = SkyCoord(ra='18 18 40.265', dec='-13 45 59.00', unit=(u.hourangle, u.degree), frame='icrs')
radius = u.Quantity(1.0, u.arcsec)
j = Gaia.cone_search_async(coord, radius)
r = j.get_results()
print r

# with open("gaia_queries.txt", "a") as myfile:
#     myfile.write(k + '\n')
#     myfile.close()
# try:
#     job = Gaia.launch_job_async(k,name=maser_list['name'][i])


    

In [75]:
ngc_6611=pd.read_csv('ngc_6611_gaia_matches.csv',delimiter="|",header=0)
pmra_ng = ngc_6611['pmra'].values
print len(np.where(pmra_ng=='--')[0])
#print np.where(pmra_ng=='--')
pmra_ng[pmra_ng=='--'] = 'nan'
pmra_ng=pmra_ng.astype(np.float)
#print len(ra_ng)
ra_ng=ngc_6611['ra'][~np.isnan(pmra_ng)]
dec_ng= ngc_6611['dec'][~np.isnan(pmra_ng)]
#print dec_ng
pmdec_ng=ngc_6611['pmdec']
pmdec_ng[pmdec_ng=='--'] = 'nan'
pmdec_ng=pmdec_ng.astype(np.float)
pmdec_ng=pmdec_ng[~np.isnan(pmra_ng)]
parallax_ng=ngc_6611['parallax']
print len(np.where(parallax_ng=='--')[0])
parallax_ng[parallax_ng=='--'] = 'nan'
parallax_ng=parallax_ng.astype(np.float)
parallax_ng=parallax_ng[~np.isnan(parallax_ng)]
pmra_ng= pmra_ng[~np.isnan(pmra_ng)]
plt.hist(parallax_ng,bins=20)
plt.show()
#print type(pmra_ng[0])
# print len(pmra_ng[20])
# pmdec_ng= ngc_6611['pmdec']#[~pmra_ng='--']
# print pmdec_ng
#pmra_ng=pmra_ng[~np.isnan(pmra)]

# plt.hist(pmra_ng,bins=20)
# plt.hist(parallax_ng,bins=20)
# plt.show()

335
335


/Users/admin/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':
/Users/admin/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [3]:
plt.plot(ra_ng,dec_ng,'.')
plt.show()

NameError: name 'ra_ng' is not defined

In [16]:
np.where(pmra_ng=='--')

(array([  16,   17,   20,   21,   70,   85,   86,  155,  157,  158,  191,
         192,  197,  198,  199,  200,  207,  208,  229,  238,  239,  271,
         348,  349,  363,  364,  365,  366,  370,  374,  377,  387,  431,
         432,  465,  466,  498,  499,  546,  547,  569,  570,  583,  584,
         600,  601,  604,  605,  663,  664,  741,  742,  773,  774,  795,
         796,  797,  837,  866,  867,  868,  878,  879,  880,  905,  906,
         907,  910,  911,  914,  915,  916,  924,  925,  935,  953,  954,
         957,  958,  959,  960,  975,  976,  987,  988, 1002, 1003, 1017,
        1018, 1022, 1023, 1032, 1034, 1035, 1055, 1056, 1059, 1062, 1063,
        1067, 1068, 1069, 1070, 1093, 1094, 1095, 1096, 1097, 1098, 1099,
        1100, 1115, 1116, 1129, 1130, 1135, 1136, 1137, 1138, 1157, 1158,
        1159, 1169, 1170, 1179, 1180, 1183, 1188, 1189, 1192, 1193, 1196,
        1197, 1198, 1199, 1205, 1206, 1207, 1213, 1216, 1217, 1218, 1219,
        1223, 1224, 1229, 1230, 1251, 

In [5]:
print parallax_ng


NameError: name 'parallax_ng' is not defined

In [97]:
Gaia.login(user='ktirimba', password='Hardenberg_49')


NameError: name 'Gaia' is not defined

In [131]:
#ngc_188_cl = pd.read_csv('2m_1306_NGC_2439.dat',header=None,delim_whitespace=True)
ngc_cl=np.loadtxt('2m_1306_NGC_2439.dat',usecols=(0,1,-4,-3,-2,-5),unpack=True)
# print ngc_cl[5,:5]
print ngc_cl[5,:]
ra_cl = ngc_cl[0,:]#[(ngc_cl[5,:]>0)& (ngc_cl[2,:]>61)]
#ra_cl = ngc_cl[0,:][np.logical_and(ngc_cl[2,:]>61, ngc_cl[3,:]>61, ngc_cl[4,:]>61)]
# print len(ra_cl)
dec_cl = ngc_cl[1,:]#[(ngc_cl[5,:]>0)&(ngc_cl[2,:]>61)]
ra_cl=SkyCoord(ra_cl*u.hourangle,dec_cl*u.deg,frame='icrs').ra.deg
#print len(ra_cl)
dec_cl = SkyCoord(ra_cl*u.hourangle,dec_cl*u.deg,frame='icrs').dec.deg
# radius = u.Quantity(1.0, u.deg)
# j = Gaia.cone_search_async(coord, radius)
# j.get_results()
print len(dec_cl)
# with open("ngc2439.csv", "a") as myfile:
#     for i in np.arange(len(ra_cl)):
#         #myfile.write(str(ngc_cl[5,:])+'\n')
#         myfile.write(str(ra_cl[i]) +','+ str(dec_cl[i])+'\n')
#print ngc_188_cl[0,1]
#print ngc_188_cl.keys()
# print np.shape(ngc_188_cl)
# print (len(ngc_188_cl))
# plt.hist(ngc_188_cl)
# plt.show()
#print len(ngc_188_cl[ngc_188_cl>75])
#prob_val=ngc_188_cl['prob. '].values
#print prob_val
#y = prob_val.astype(np.float)

#print ngc_188_cl['prob. ']

[1. 1. 1. ... 0. 0. 0.]
20398


In [6]:
Gaia.login(user='ktirimba', password='Hardenberg_49')
table = 'ngc2439.csv'
Gaia.upload_table(upload_resource=table, table_name='ngc2439')

full_qualified_table_name = 'user_tirimba.my_sources'
xmatch_table_name = 'xmatch_table'
Gaia.cross_match(full_qualified_table_name_b='gaiadr2.gaia_source',results_table_name=xmatch_table_name, radius=0.000555)

# # Once you have your cross match finished, you can obtain the results:
# xmatch_table = 'user_<your_login_name>.' + xmatch_table_name
# query = 'SELECT c."dist", a.*, b.* FROM gaiadr2.data_source AS a, '+\
# full_qualified_table_name+' AS b, '+\
# xmatch_table+' AS c '+\
# 'WHERE (c.gaia_source_source_id = a.source_id AND c.my_sources_my_sources_oid = b.my_sources_oid)'
# job = Gaia.launch_job(query=query)
# results = job.get_resultsjob)
# print(results)

OK
Sending file: ngc2439.csv
(500, 'Error 500:\nesavo.tap.TAPException: Error while uploading table "ngc2439" : Content is not allowed in prolog.')


HTTPError: Error 500:
esavo.tap.TAPException: Error while uploading table "ngc2439" : Content is not allowed in prolog.

In [8]:
def cross_check(ra_cand,dec_cand,cluster_file):
    percent_thresh=np.array([])
    ngc_cl=np.loadtxt(cluster_file,usecols=(0,1,-4,-3,-2,-5),unpack=True)
    ra_cl = ngc_cl[0,:]#[(ngc_cl[5,:]>0)& (ngc_cl[2,:]>61)]
    dec_cl = ngc_cl[1,:]#[(ngc_cl[5,:]>0)&(ngc_cl[2,:]>61)]
    ra_cl=SkyCoord(ra_cl*u.hourangle,dec_cl*u.deg,frame='icrs').ra.deg
    #print len(ra_cl)
    dec_cl = SkyCoord(ra_cl*u.hourangle,dec_cl*u.deg,frame='icrs').dec.deg
    for cand in np.arange(len(ra_cand)):
        #print ra_cand[0][cand],dec_cand[0][cand]
    #     diff=np.sqrt((ra_cl-ra_cand[0][cand])**2+(dec_cl-dec_cand[0][cand])**2)
        diff=np.degrees(np.arccos(np.sin(np.radians(dec_cl))*np.sin(np.radians(dec_cand[cand]))+\
        np.cos(np.radians(dec_cl))*np.cos(np.radians(dec_cand[cand]))*np.cos(np.radians(ra_cl-ra_cand[cand]))))
        #diff_ra=np.abs(ra_cl-ra_cand[0][cand])
        if np.min(diff)<0.000555556:
            index=np.argmin(diff)
    #         print ngc_cl[2,:][index]
            percent_thresh=np.append(percent_thresh,ngc_cl[2,:][index])
    return np.sum(percent_thresh>65)/len(percent_thresh)       

In [47]:
# np.sum(percent_thresh>68)/len(percent_thresh)#/len(percent_thresh)
maser_list = pd.read_csv('/Users/admin/Downloads/Master_project/masers.dat', sep=' ', header=0)
maser_ra,maser_dec=maser_list['ra'][maser_list['arm']=='Per'], maser_list['dec'][maser_list['arm']=='Per']

In [49]:
coords = SkyCoord(ra=maser_ra,dec=maser_dec, unit=(u.hourangle, u.deg))
galactic=coords.galactic
plt.plot(galactic.ra,galactic.d)

<SkyCoord (ICRS): (ra, dec) in deg
    [(282.18783667, -1.55366667e+00), (282.62805417, -3.31333333e-02),
     (285.66508   ,  6.98584778e+00), (286.50678667,  6.77670556e+00),
     (286.86743083,  8.18150917e+00), (287.555875  ,  9.10355556e+00),
     (290.1299    ,  1.39236694e+01), (290.18690458,  1.46407956e+01),
     (290.2467075 ,  1.47804481e+01), (296.46872875,  2.42953436e+01),
     (324.99279208,  5.02391650e+01), (324.91878708,  5.13424467e+01),
     (325.756     ,  5.49382778e+01), (334.04318792,  5.23594758e+01),
     (342.38115625,  5.99283350e+01), (342.49531667,  6.02990694e+01),
     (345.63366667,  5.69642778e+01), (343.15964583,  6.00144133e+01),
     (344.37419167,  6.24963436e+01), (349.33662   ,  5.94797139e+01),
     (349.04309167,  5.99246100e+01), (348.439     ,  6.14695972e+01),
     (356.01367333,  6.17894964e+01), ( 11.24332083,  5.57798889e+01),
     ( 13.10292   ,  5.65640353e+01), ( 13.10081667,  5.65619944e+01),
     ( 36.7659125 ,  6.18736750e+01), ( 35

In [32]:
# 0, 0.35294117647058826
# 1,
# 2, 0.4715909090909091
# 4, 0.609254498714653
plt.bar([0,1,2,3,4,5],[0.35,0.5,0.47,0.3,0.61,0.5],color='b',label='DBSCAN')
plt.bar([0.05,1.05,2.05,3.05,4.05,5.05],[0.287,0.218,0.438,0.3,0.64,0.22],color='y',label='D-Value')
plt.bar([0.1,1.1,2.1,3.1,4.1,5.1],[0.09,0.12,0.08,0.10,0.19,0.09],color='r',label='HDBSCAN')
plt.ylabel('N$_{conf}$')
plt.legend()
plt.show()

In [38]:
maser_list = pd.read_csv('~/Downloads/Master_project/masers.dat', sep=' ', header=0)
arms=maser_list['arm']
kra=maser_list['ra'][arms=='Per'].array
kdec=maser_list['dec'][arms=='Per'].array
kpar = maser_list['parallax'][arms=='Per'].array
kpar_error = maser_list['parallax_error'][arms=='Per'].array
with open("perseus_masers.csv", "a") as myfile:
    for n in np.arange(len(kra)):
        myfile.write(str(kra[n]) +','+ str(kdec[n])+','+ str(kpar[n])+','+str(kpar_error[n])+'\n')
